[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Jonarod/Bootcamp-Inteligencia-Artificial-Women-Who-Code-Medellin/blob/master/semana%205/ChatBot%20Simple%20con%20Embeddings/ChatBot_Simple_con_Embeddings.ipynb)

In [83]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import json
import numpy as np
print(tf.__version__)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2.8.0


In [48]:
import sys
print(sys.executable)

/usr/bin/python3


In [49]:
oov_tok = '<OOV>'

In [50]:
from google.colab import files
files.upload()

{}

In [51]:
with open('/content/intents.json', 'r') as f:
    intents = json.load(f)

In [108]:
# Funcion para sacar acentos (util para el Espanol, usar con cuidado segun el idioma)
import unicodedata
import string

def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def strip_punctuation(s):
   return s.translate(str.maketrans('', '', string.punctuation + "¿" + "¡"))

In [124]:
intent_list = []
label_list = []

# ignore_words = stopwords.words('spanish')
# print('Stop Words:\n', ignore_words)

# print(ignore_words)
# stemmer = SnowballStemmer("spanish")


for index, intent in enumerate(intents['intents']):
  intent_list +=  [strip_punctuation(strip_accents(w.lower())) for w in intent['patterns']]
  num_patterns = len(intent['patterns'])
  label_list += [index] * num_patterns

print('Intents:\n', intent_list)
print('Labels:\n', label_list)


tokenizer = Tokenizer(oov_token = oov_tok)
tokenizer.fit_on_texts(intent_list)
word_index = tokenizer.word_index

print('Diccionario:\n', word_index)


sequences = tokenizer.texts_to_sequences(intent_list)
print('Sequences:\n', sequences)

Intents:
 ['hola', 'como estas', 'como estas tu', 'que tal', 'buenos dias', 'adios', 'hasta luego', 'gracias', 'mucho gracias', 'eso es util', 'que articulos tienes', 'que tipo de articulos hay', 'que vendes', 'aceptan tarjetas de credito', 'aceptan mastercard', 'puedo pagar con paypal', 'es solo efectivo', 'cuanto tiempo tarda la entrega', 'cuanto tiempo tarda el envio', 'cuando recibo mi entrega', 'cuentame un chiste', 'dime algo gracioso', 'conoces un chiste']
Labels:
 [0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 6, 6, 6]
Diccionario:
 {'<OOV>': 1, 'que': 2, 'como': 3, 'estas': 4, 'gracias': 5, 'es': 6, 'articulos': 7, 'de': 8, 'aceptan': 9, 'cuanto': 10, 'tiempo': 11, 'tarda': 12, 'entrega': 13, 'un': 14, 'chiste': 15, 'hola': 16, 'tu': 17, 'tal': 18, 'buenos': 19, 'dias': 20, 'adios': 21, 'hasta': 22, 'luego': 23, 'mucho': 24, 'eso': 25, 'util': 26, 'tienes': 27, 'tipo': 28, 'hay': 29, 'vendes': 30, 'tarjetas': 31, 'credito': 32, 'mastercard': 33, 'puedo': 34, 'pag

In [125]:
max_length = len(max(sequences, key=len))
vocab_size = len(word_index)
padded = pad_sequences(sequences)
padded_length = len(padded[0])
labels = tf.keras.utils.to_categorical(label_list)
num_categories = len(labels[0])

In [161]:
num_epochs = 300

model = tf.keras.Sequential([
    layers.Embedding(vocab_size+1, 24, input_length=padded_length),
    layers.Flatten(),
    layers.Dense(24, activation='relu'),
    layers.Dense(24, activation='relu'),
    layers.Dense(num_categories, activation='softmax')
])
model.compile( loss='categorical_crossentropy', optimizer='adam', metrics=['acc'] )


model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, 5, 24)             1224      
                                                                 
 flatten_19 (Flatten)        (None, 120)               0         
                                                                 
 dense_51 (Dense)            (None, 24)                2904      
                                                                 
 dense_52 (Dense)            (None, 24)                600       
                                                                 
 dense_53 (Dense)            (None, 7)                 175       
                                                                 
Total params: 4,903
Trainable params: 4,903
Non-trainable params: 0
_________________________________________________________________


In [162]:
history = model.fit(padded, labels, epochs=num_epochs, verbose=2)

Epoch 1/300
1/1 - 1s - loss: 1.9422 - acc: 0.1739 - 555ms/epoch - 555ms/step
Epoch 2/300
1/1 - 0s - loss: 1.9353 - acc: 0.3043 - 7ms/epoch - 7ms/step
Epoch 3/300
1/1 - 0s - loss: 1.9289 - acc: 0.3478 - 7ms/epoch - 7ms/step
Epoch 4/300
1/1 - 0s - loss: 1.9224 - acc: 0.3478 - 8ms/epoch - 8ms/step
Epoch 5/300
1/1 - 0s - loss: 1.9159 - acc: 0.3478 - 9ms/epoch - 9ms/step
Epoch 6/300
1/1 - 0s - loss: 1.9094 - acc: 0.4348 - 8ms/epoch - 8ms/step
Epoch 7/300
1/1 - 0s - loss: 1.9026 - acc: 0.4348 - 10ms/epoch - 10ms/step
Epoch 8/300
1/1 - 0s - loss: 1.8959 - acc: 0.3913 - 7ms/epoch - 7ms/step
Epoch 9/300
1/1 - 0s - loss: 1.8892 - acc: 0.3913 - 8ms/epoch - 8ms/step
Epoch 10/300
1/1 - 0s - loss: 1.8822 - acc: 0.3913 - 8ms/epoch - 8ms/step
Epoch 11/300
1/1 - 0s - loss: 1.8749 - acc: 0.3913 - 7ms/epoch - 7ms/step
Epoch 12/300
1/1 - 0s - loss: 1.8674 - acc: 0.3913 - 8ms/epoch - 8ms/step
Epoch 13/300
1/1 - 0s - loss: 1.8596 - acc: 0.3913 - 8ms/epoch - 8ms/step
Epoch 14/300
1/1 - 0s - loss: 1.8516 - ac

In [163]:
sentence = ['hola', 'cual tarjetas aceptan', 'adios']

def infer(sentence):
  sequence = tokenizer.texts_to_sequences([sentence])
  padded_sequence = pad_sequences(sequence, maxlen=padded_length)
  prediction = model.predict(padded_sequence)[0]
  intent = intents['intents'][np.argmax(prediction)]
  tag = intent['tag']
  return sentence, tag, prediction


for s in sentence:
  sentence, tag, prediction = infer(s)
  print(sentence, tag, prediction)

hola saludo [9.9616241e-01 2.4719799e-03 1.0263220e-03 1.0493292e-06 2.2839858e-04
 1.0954233e-04 3.5792888e-07]
cual tarjetas aceptan pagos [0.27704763 0.02086235 0.11658002 0.05745263 0.37267935 0.05740475
 0.09797332]
adios adiós [2.7737217e-03 9.9195194e-01 4.1270494e-03 6.2227709e-04 3.6324529e-04
 1.6143927e-04 5.2064871e-07]


In [ ]:
model_json = model.to_json()
with open('/content/model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('/content/model.h5')
json_file.close()

model.save('/content/saved_model')


with open('/content/wordIndex.json', 'w') as json_file:
    json.dump(word_index, json_file)
json_file.close()